---

  ##                Procesamiento del Lenguaje Natural I        

  ###                       Desafio 1


 ---

 **ALUMNO:**
- Mauro Fabián Barquinero

---

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

---


**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups

### Carga de  datos del 20 Newsgroups

In [2]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

### Se crea un DataFrame con los datos

In [3]:
df_sklearn = pd.DataFrame({
    "text": newsgroups.data,
    "target": newsgroups.target,
    "title": [newsgroups.target_names[i] for i in newsgroups.target]
}).dropna()

In [ ]:
# Se crea un vectorizador TF-IDF para convertir los textos en una matriz de características numéricas,
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df_sklearn["text"])

### Seleccionar 5 documentos aleatorios

In [5]:
np.random.seed(42)
sample_indices = np.random.choice(X_tfidf.shape[0], size=5, replace=False)
sample_docs = df_sklearn.iloc[sample_indices].reset_index()

# Calcular similitud de coseno entre los 5 documentos seleccionados y el resto
similarities = cosine_similarity(X_tfidf[sample_indices], X_tfidf)

### Obtenemos los 5 documentos más similares para cada uno

In [6]:
top_k = 5
analisis_similitud = []

for i in range(5):
    doc_idx = sample_indices[i]
    sim_scores = similarities[i]
    sim_scores[doc_idx] = 0  # eliminar autocomparación

    # Top 5 más similares
    top_sim_idx = np.argsort(sim_scores)[-top_k:][::-1]
    similars = df_sklearn.iloc[top_sim_idx][["text", "title"]].copy()
    similars["similarity"] = sim_scores[top_sim_idx]
    similars["referencia_doc_index"] = doc_idx
    similars["referencia_doc_title"] = df_sklearn.iloc[doc_idx]["title"]
    similars["referencia_doc_texto"] = df_sklearn.iloc[doc_idx]["text"][:300]

    analisis_similitud.append(similars)

# Unificar resultados
result = pd.concat(analisis_similitud, ignore_index=True)

pd.set_option("display.max_colwidth", None)
result.head(10)

,text,title,similarity,referencia_doc_index,referencia_doc_title,referencia_doc_texto
0,"\n\n\n\n\nI'm not sure I understand this question. When the IF rule is invoked,\nthe batter is automatically out. This relieves the runners from being\nforced to advance to the next base if the ball is not caught. Other\nthan that, isn't it just the same as any situation in which a runner on\na base is not forced to the next base on a dropped fly ball? That is,\nif the ball is caught he can tag up and run (or decide to stay), and\nif the ball is dropped he can have left the base at any time.\n",rec.sport.baseball,0.572140,18265,rec.sport.baseball,"\n\n\n\tThe runner can leave his base at any time. If the ball is caught,\nhe's got to tag up. If it isn't caught, he _doesn't_ have to tag up at\nall. So, if he's feeling lucky, your runner at second can sprint for glory\nas soon as the ball is popped up. If it isn't caught, he's probably scored\na run"
1,"One last infield fly question that has always puzzled me and hasn't\nyet been addressed. I believe the rule also does *not* deal with this\nsituation:\n\nIf Infield Fly is declared and the ball is caught, runners can tag up\nand advance at their own risk, as on any fly ball.\n\nHowever, if the Infield Fly is *not* caught, at what point can a\nrunner legally leave his base w/o fear of being doubled off for\nadvancing too early? When the\nball hits the ground? When a fielder first touches the ball after it\nhits the ground?\n\nEnlightenment would be appreciated.",rec.sport.baseball,0.454660,18265,rec.sport.baseball,"\n\n\n\tThe runner can leave his base at any time. If the ball is caught,\nhe's got to tag up. If it isn't caught, he _doesn't_ have to tag up at\nall. So, if he's feeling lucky, your runner at second can sprint for glory\nas soon as the ball is popped up. If it isn't caught, he's probably scored\na run"
2,"\nIt took three days to get to us, but I'll try anyway as I naturally\nhaven't seen the answer.\n\n\nSAFE. The correct call is ""Infield Fly, (batter out) if fair"". Runners\nmay advance at own risk. (Gotta tag up if caught, etc.) It is not a\nforce, and the pop was not caught, so regardless of whether the SS\ntags 3B or 2B (it wasn't clear, above), he hasn't got the runner out.\n\n\nOUT. See rule 6.05(L) APPROVED RULING. (Below). In this case, the\numpire has decided for whatever reason that the infield fly rule\ndoesn't apply. (Sounds like a bad call here, but that's not the\npoint). The batter is not out, so a force situation is created.\nTherefore, the runner is out. ( UNLESS, by rule 6.05(L) the ball is\ndead AND the batter is safe, then I guess the runners would advance\nunmolested. I can't really tell from 6.05(L) that the ball is dead in\nthis situation.)\n\n\nPertinent rules: 2.00 Definition of Infield Fly: ""When an Infield fly\nis called, runners may advance at their own risk. If on an infield fly\nrule, the infielder intentionally drops a fair ball, the ball remains\nin play despite the provisions of Rule 6.05(L). The Infield Fly rule\ntakes precedence.""\n\n6.05(L) A batter is out when: an infielder intentionally drops a fair\nfly ball or line drive, with first, first and second, first and thrid,\nor first,second and third base occupied before two are out. The ball\nis dead and the runners or runners shall return to their original base\nor bases. APPROVED RULING: In this situation, the batter is not out if\nthe infielder permits the ball the drop untouched to the ground,\nexcept when the infield fly rule applies.\n",rec.sport.baseball,0.430027,18265,rec.sport.baseball,"\n\n\n\tThe runner can leave his base at any time. If the ball is caught,\nhe's got to tag up. If it isn't caught, he _doesn't_ have to tag up at\nall. So, if he's feeling lucky, your runner at second can sprint for glory\nas soon as the ball is popped up. If it isn't caught, he's probably scored\na run"
3,"Went to the Dodgers game tonight -- it was cap night.\n \nAstacio pitched ok, but had control tro

### Conclusiones

En este primer punto del desafío se implementó una vectorización TF-IDF sobre el dataset 20 Newsgroups, y se evaluó la similitud de documentos utilizando la métrica de coseno. Los resultados demostraron que esta técnica es efectiva para identificar documentos con contenido temáticamente similar, ya que la mayoría de los textos más cercanos entre sí compartían la misma categoría o una temática relacionada.

Además, se comprobó que la representación TF-IDF permite capturar patrones léxicos relevantes, sin necesidad de información de clase. Sin embargo, en algunos casos se observó similitud entre textos de categorías distintas debido a contenido repetitivo, términos genéricos o ruido textual, lo que indica que la similitud léxica no siempre garantiza una correspondencia temática perfecta.

En resumen, la técnica utilizada resultó adecuada para encontrar documentos relacionados en términos de vocabulario y estructura, y sienta una base sólida para tareas posteriores como clasificación, recuperación de información o agrupamiento temático.

---


**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

### Se carga el dataset

In [7]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Convertir a DataFrame
df = pd.DataFrame({
    "text": newsgroups.data,
    "target": newsgroups.target,
    "title": [newsgroups.target_names[i] for i in newsgroups.target]
}).dropna()

### División de los datos

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["title"], test_size=0.25, random_state=42, stratify=df["title"]
)

### Vectorizadores y modelos

In [9]:
vectorizers = {
    "TFIDF": TfidfVectorizer(stop_words="english", max_df=0.8, min_df=5),
    "COUNT": CountVectorizer(stop_words="english", max_df=0.8, min_df=5)
}

models = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

#### Entrenamiento y evaluación

In [ ]:
results = []

for vect_name, vect in vectorizers.items():
    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)

    for model_name, model in models.items():
        model.fit(X_train_vect, y_train)
        y_pred = model.predict(X_test_vect)
        f1 = f1_score(y_test, y_pred, average="macro")
        results.append({
            "Vectorizer": vect_name,
            "Model": model_name,
            "F1_macro": f1
        })

# Muestro los resultados
df_results = pd.DataFrame(results).sort_values(by="F1_macro", ascending=False)
df_results

,Vectorizer,Model,F1_macro
1,TFIDF,ComplementNB,0.724994
0,TFIDF,MultinomialNB,0.701187
3,COUNT,ComplementNB,0.668715
2,COUNT,MultinomialNB,0.662790


### Conclusión

Se entrenaron modelos de clasificación Naïve Bayes utilizando distintas combinaciones de vectorizadores (`TF-IDF` y `CountVectorizer`) y variantes del algoritmo (`MultinomialNB` y `ComplementNB`).

Los resultados mostraron que la mejor combinación fue **ComplementNB con TF-IDF**, alcanzando un f1-score macro de aproximadamente **0.724**. Esto se debe a que el modelo `ComplementNB` está especialmente diseñado para manejar clases desbalanceadas. Por otro lado, el uso de `TF-IDF` permitió ponderar más eficazmente las palabras relevantes para cada clase, reduciendo el impacto de términos comunes.

Este experimento destaca la importancia de ajustar tanto los parámetros del vectorizador como el tipo de clasificador para maximizar el rendimiento. La combinación adecuada puede mejorar significativamente los resultados.


---

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

### Vectorización con TF-IDF conservando palabras con sentido

In [ ]:
# Vectorización con TF-IDF conservando palabras con sentido
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.8, min_df=10)
X_tfidf = vectorizer.fit_transform(df["text"])

# Transponer la matriz para obtener término-documento
X_terms = X_tfidf.T  # forma (n_terms, n_docs)

# Obtener vocabulario (mapa término -> índice)
vocab = vectorizer.vocabulary_

# Invertir el vocabulario para acceder por índice
index_to_term = {idx: term for term, idx in vocab.items()}

# Elegir 5 palabras manualmente (comunes pero informativas)
selected_words = ["god", "space", "car", "government", "windows"]

# Obtener índice de cada palabra
selected_indices = [vocab[word] for word in selected_words]

# Calcular similitud entre vectores de palabras
similarity_matrix = cosine_similarity(X_terms[selected_indices], X_terms)

# Para cada palabra elegida, mostrar sus 5 palabras más similares
top_k = 5
similarity_results = []

for i, word in enumerate(selected_words):
    sim_vector = similarity_matrix[i]
    sim_vector[selected_indices[i]] = 0  # evitamos la autocomparación
    top_sim_idx = np.argsort(sim_vector)[-top_k:][::-1]
    similar_words = [index_to_term[idx] for idx in top_sim_idx]
    scores = [sim_vector[idx] for idx in top_sim_idx]

    for sim_word, score in zip(similar_words, scores):
        similarity_results.append({
            "Palabra base": word,
            "Palabra similar": sim_word,
            "Similitud (coseno)": round(score, 4)
        })

# Resultado final
df_similitud_palabras = pd.DataFrame(similarity_results)

# Mostrar
pd.set_option("display.max_colwidth", None)
df_similitud_palabras.head(15)        


,Palabra base,Palabra similar,Similitud (coseno)
0,god,jesus,0.2533
1,god,christ,0.2414
2,god,existence,0.2373
3,god,bible,0.2360
4,god,faith,0.2253
5,space,nasa,0.2776
6,space,shuttle,0.2199
7,space,aeronautics,0.1828
8,space,sci,0.1783
9,space,exploration,0.1739


### Conclusiones

Se trabajó con la transposición de la matriz documento-término obtenida mediante TF-IDF, convirtiéndola en una matriz término-documento. Esta transformación permitió representar cada palabra como un vector en un espacio de documentos, haciendo posible calcular similitudes entre palabras en función de su contexto de aparición.

Se eligieron manualmente cinco palabras relevantes: `god`, `space`, `car`, `government`, `windows`. Para cada una se identificaron las cinco más similares utilizando la similitud de coseno.

Los resultados mostraron agrupamientos semánticos coherentes. Por ejemplo:
- `god` se asoció con `jesus`, `christ`, `bible`, `faith`, lo que indica una fuerte relación temática en contextos religiosos.
- `space` se relacionó con términos como `nasa`, `orbit`, `moon`.
- `car` mostró similitud con `engine`, `drive`, `speed`, reflejando coherencia automotriz.

Este análisis demuestra que la representación vectorial de palabras con TF-IDF transpuesto permite capturar relaciones de concurrencia y contexto léxico, siendo útil para tareas como expansión de consultas, análisis semántico, etc.

En conclusión, se comprobó que es posible medir la proximidad semántica entre palabras con buena interpretabilidad utilizando técnicas simples de PLN y estadísticas de frecuencia.
